<a href="https://colab.research.google.com/github/LuizHVicari/Sistemas-Inteligentes/blob/main/HW9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [175]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [176]:
lista_maca_peso = [168, 201, 184, 216, 177, 230, 149, 192, 206, 219, 177, 194, 241, 162, 207]
lista_maca_diam = [7.7, 9.2, 8.5, 9.8, 8.1, 10.1, 6.9, 8.8, 9.4, 10.0, 8.2, 9.0, 10.6, 7.5, 9.6]
maca = ['maca' for i in range(15)]

lista_laranja_peso = [150, 162, 170, 182, 195, 204, 213, 221, 234, 246, 257, 266, 278, 287, 296]
lista_laranja_diam = [6, 6.5, 6.8, 7.1, 7.4, 7.7, 8, 8.3, 8.6, 8.9, 9.2, 9.5, 9.8, 10.1, 10.4]
laranja = ['laranja' for i in range(15)]

peso = lista_maca_peso + lista_laranja_peso
diam = lista_maca_diam + lista_laranja_diam
frutas = maca + laranja

df = pd.DataFrame(columns=['peso', 'diametro', 'fruta'])
df['peso'] = peso
df['diametro'] = diam
df['fruta'] = frutas

df['fruta'] = df.apply(lambda x : 0 if x['fruta'] == 'maca' else 1, axis=1)
# y = np.column_stack([y])

In [177]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['fruta']), df['fruta'])

In [178]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [179]:
class Particle():
    def __init__(self, pb_importance, nb_importance):
        self.pb_importance = pb_importance
        self.nb_importance = nb_importance
        self.velocity = np.array([0, 0, 0])
        self.position = np.array([np.random.randint(-1000, 1000), np.random.randint(-1000, 1000), np.random.randint(-1000, 1000)]) / 1000
        self.pb = self.position
        self.pb_score = 0

    def update_velocity(self, gbest):
        try:
            rand = np.random.randint(-1000, 1000) / 1000
            self.velocity[0] = self.velocity[0] + self.pb_importance * rand * self.pb[0] + self.nb_importance * rand * gbest[0]
            self.velocity[1] = self.velocity[1] + self.pb_importance * rand * self.pb[1] + self.nb_importance * rand * gbest[1]
            self.velocity[2] = self.velocity[2] + self.pb_importance * rand * self.pb[2] + self.nb_importance * rand * gbest[2]
        except OverflowError:
            pass

    def update_position(self):
        self.position += self.velocity

    def update_personal_best(self, new_score, new_position):
        if new_score > self.pb_score:
            self.pb_score = new_score
            self.pb = new_position

    def predict(self, X):
        y = X[0] * self.position[0] + X[1] * self.position[1] + self.position[2]
        return 0 if y < 0 else 1

In [180]:
class PSO():
    def __init__(self, n_particles : int = 20, pb_importance : float = 1.0, nb_importance : float = 3.0, n_iterations : int = 100):
        self.particles = [Particle(pb_importance, nb_importance) for _ in range(n_particles)]
        self.n_particles = n_particles
        self.g_best = None
        self.g_best_score = 0
        self.best_particle = None
        self.n_iterations = n_iterations

    def fit(self, X_train, y_train):
        best_score = 0
        for _ in range(self.n_iterations):
            p_score = list()
            for i, particle in enumerate(self.particles):
                y_pred = list()
                for line in X_train:
                    y_pred.append(particle.predict(line))
                y_pred = np.array(y_pred)

                p_score.append(self.score(y_train, y_pred))
                particle.update_personal_best(p_score[i], particle.position)

            g_best_score = max(p_score)
            g_best_pos = p_score.index(g_best_score)

            if g_best_score > self.g_best_score:
                self.g_best_score = g_best_score
                self.best_particle = self.particles[g_best_pos]
                self.g_best = self.best_particle.position

                if self.g_best_score == 1.0:
                    break

            for i, particle in enumerate(self.particles):
                particle.update_velocity(self.g_best)
                particle.update_position()

    def score(self, y, y_pred):
        # print(y)
        # print(y_pred)
        # print(y_pred.shape)
        return (np.sum(y == y_pred) / y_pred.shape)[0]

    def predict(self, X):
        if self.best_particle != None:
            y_pred = list()
            for line in X:
                y_pred.append(self.best_particle.predict(line))
            return np.array(y_pred)
        else:
            raise Exception("Não há partículas treinadas")

pso = PSO(n_iterations = 100)
pso.fit(X_train, y_train)
pred = pso.predict(X_test)
pso.score(pred, y_test)

1.0

In [181]:
pso.best_particle.position

array([ 4.709, -3.223,  0.756])

In [182]:
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore')

rs = ShuffleSplit(n_splits=5, test_size=0.33, random_state=1)

train_score = np.array([])
test_score = np.array([])

for i,(train_data, test_data)in enumerate(rs.split(df.drop(columns=['fruta']))):
    print(f'Iteração {i+1}')
    train_data_i = df.query(' index in @train_data')
    X_train_i = train_data_i.drop(columns=['fruta'])
    y_train_i = train_data_i['fruta']

    test_data_i = df.query('index in @test_data')
    X_test_i = test_data_i.drop(columns=['fruta'])
    y_test_i = test_data_i['fruta']

    X_train_i, X_test_i = X_train_i.to_numpy(), X_test_i.to_numpy()


    pso = PSO(n_iterations=100)

    pso.fit(X_train_i, y_train_i)

    y_pred_train_i = pso.predict(X_train_i)
    y_pred_test_i = pso.predict(X_test_i)

    train_score_i = pso.score(y_pred_train_i, y_train_i)
    test_score_i = pso.score(y_pred_test_i, y_test_i)

    train_score = np.append(train_score, train_score_i)
    test_score = np.append(test_score, test_score_i)

    print(f'Precisão na base de treino: {train_score_i}')
    print(f'Precisão na base de teste: {test_score_i}')

print(f'\nPrecisão média:\n\tTeste: {train_score.mean()}\n\tTreino: {test_score.mean()}')
print(f'STD médio:\n\tTeste: {train_score.std()}\n\tTreino: {test_score.std()}')

Iteração 1
Precisão na base de treino: 1.0
Precisão na base de teste: 1.0
Iteração 2
Precisão na base de treino: 0.5
Precisão na base de teste: 0.5
Iteração 3
Precisão na base de treino: 0.55
Precisão na base de teste: 0.4
Iteração 4
Precisão na base de treino: 0.5
Precisão na base de teste: 0.5
Iteração 5
Precisão na base de treino: 0.55
Precisão na base de teste: 0.4

Precisão média:
	Teste: 0.6199999999999999
	Treino: 0.5599999999999999
STD médio:
	Teste: 0.19131126469708992
	Treino: 0.22449944320643647
